In [ ]:
# Importation des bibliothèques nécessaires
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Chargement des données MNIST
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Normalisation des données
x_train, x_test = x_train / 255.0, x_test / 255.0

# Redimensionnement des données pour qu'elles correspondent au format attendu par les couches Conv2D
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)

# Création du modèle
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

# Compilation du modèle
model.compile(loss=tf.keras.losses.sparse_categorical_crossentropy,
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])

# Create an ImageDataGenerator object
datagen = ImageDataGenerator(
    rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
    zoom_range = 0.1, # Randomly zoom image
    width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
)

# Compute quantities required for feature-wise normalization
datagen.fit(x_train)

# Entraînement du modèle
model.fit(datagen.flow(x_train, y_train, batch_size=128),
          validation_data=(x_test, y_test),
          epochs=10, verbose=1)

# Évaluation du modèle
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

11490434/11490434 [==============================] - 2s 0us/step
Epoch 1/10
469/469 [==============================] - 33s 46ms/step - loss: 0.4853 - accuracy: 0.8454 - val_loss: 0.0492 - val_accuracy: 0.9834
Epoch 2/10
469/469 [==============================] - 20s 43ms/step - loss: 0.1887 - accuracy: 0.9421 - val_loss: 0.0343 - val_accuracy: 0.9887
Epoch 3/10
469/469 [==============================] - 20s 42ms/step - loss: 0.1533 - accuracy: 0.9550 - val_loss: 0.0317 - val_accuracy: 0.9884
Epoch 4/10
469/469 [==============================] - 19s 40ms/step - loss: 0.1297 - accuracy: 0.9610 - val_loss: 0.0293 - val_accuracy: 0.9903
Epoch 5/10
469/469 [==============================] - 19s 40ms/step - loss: 0.1167 - accuracy: 0.9642 - val_loss: 0.0304 - val_accuracy: 0.9896
Epoch 6/10
469/469 [==============================] - 18s 38ms/step - loss: 0.1089 - accuracy: 0.9678 - val_loss: 0.0248 - val_accuracy: 0.9922
Epoch 7/10
469/469 [==============================] - 19s 40ms/step - l

In [ ]:
from IPython.display import HTML, Image
from google.colab.output import eval_js
from base64 import b64decode

canvas_html = """
<canvas width=%d height=%d></canvas>
<button>Finish</button>
<script>
var canvas = document.querySelector('canvas')
var ctx = canvas.getContext('2d')
ctx.fillStyle = "#FFFFFF";
ctx.fillRect(0, 0, canvas.width, canvas.height);
ctx.lineWidth = %d
var button = document.querySelector('button')
var mouse = {x: 0, y: 0}
canvas.addEventListener('mousemove', function(e) {
  mouse.x = e.pageX - this.offsetLeft
  mouse.y = e.pageY - this.offsetTop
})
canvas.onmousedown = ()=>{
  ctx.beginPath()
  ctx.moveTo(mouse.x, mouse.y)
  canvas.addEventListener('mousemove', onPaint)
}
canvas.onmouseup = ()=>{
  canvas.removeEventListener('mousemove', onPaint)
}
var onPaint = ()=>{
  ctx.rect(Math.floor(mouse.x/%d)*%d, Math.floor(mouse.y/%d)*%d, %d, %d)
  ctx.stroke()
}
var data = new Promise(resolve=>{
  button.onclick = ()=>{
    resolve(canvas.toDataURL('image/png'))
  }
})
</script>
"""

def draw(filename='drawing.png', w=400, h=200, line_width=1):
  display(HTML(canvas_html % (w, h, line_width, line_width, line_width, line_width, line_width, line_width, line_width)))
  data = eval_js("data")
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return len(binary)

In [ ]:
from tensorflow import keras
from tensorflow.keras.preprocessing import image
import numpy as np
from PIL import Image
from PIL import ImageOps

def preprocess_image(img):
    # Convert the image to grayscale
    img = img.convert("L")

    # Resize the image
    img = img.resize((28, 28), Image.NEAREST)

    # Convert the image to a numpy array
    img = np.array(img)

    # Invert the colors
    img = 255 - img

    # Scale the image data to the range [0, 1]
    img = img / 255.0

    # Reshape the image data for use in the model
    img = img.reshape(1, 28, 28, 1)

    return img

def predict_image(filename):
    # Load the image
    img = Image.open(filename)

    # Preprocess the image
    img = preprocess_image(img)

    # Use the model to make a prediction
    predictions = model.predict(img)

    # Get the top 3 predictions and their confidence levels
    top3_predictions = np.argsort(predictions[0])[-3:][::-1]
    top3_confidences = predictions[0][top3_predictions]

    return list(zip(top3_predictions, top3_confidences))

In [ ]:
from IPython.display import clear_output

for i in range(20):
    filename = f"test_{i}.png"
    draw(filename=filename, w=400, h=400, line_width=10)

    predicted_digit = predict_image(filename)
    print(f"Predictions for {filename}:")
    for i, (digit, confidence) in enumerate(predicted_digit):
        print(f"  #{i+1}: Digit {digit} with confidence {confidence * 100:.2f}%")

    while True:
        try:
            print("If the model is incorrect, please enter the correct label: ", end='\n')
            correct_label = int(input())
            break
        except ValueError:
            print("That's not a valid number. Please enter again.")

    image = Image.open(filename)

    image = preprocess_image(image)

    # Add the new example to the training data
    x_train = np.concatenate([x_train, image])
    y_train = np.concatenate([y_train, [correct_label]])

    clear_output(wait=True)

# After collecting all the new data, retrain the model on the updated data
# You might want to use a smaller learning rate for this
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train, epochs=5)

# Save the model so that the training last :
model.save('model_after_training.keras')


1/1 [==============================] - 0s 17ms/step
Predictions for test_5.png:
  #1: Digit 9 with confidence 95.88%
  #2: Digit 4 with confidence 1.52%
  #3: Digit 3 with confidence 1.27%
If the model is incorrect, please enter the correct label: 


In [ ]:
# Load the trained model

from tensorflow.keras.models import load_model
model = load_model('model_after_training.h5')

for i in range(20):
    filename = f"test_{i}.png"
    draw(filename=filename, w=400, h=400, line_width=10)

    predicted_digit = predict_image(filename)
    print(f"Predictions for {filename}:")
    for i, (digit, confidence) in enumerate(predicted_digit):
        print(f"  #{i+1}: Digit {digit} with confidence {confidence * 100:.2f}%")

    while True:
        try:
            print("If the model is incorrect, please enter the correct label: ", end='\n')
            correct_label = int(input())
            break
        except ValueError:
            print("That's not a valid number. Please enter again.")

    image = Image.open(filename)

    image = preprocess_image(image)

    # Add the new example to the training data
    x_train = np.concatenate([x_train, image])
    y_train = np.concatenate([y_train, [correct_label]])

    clear_output(wait=True)




OSError: No file or directory found at model_after_training.h5